# Ozone AQI
 Model Fit - Weekly
Fitting Appropiate Model To data

__Setup Enviroment__

In [1]:
# Install Packages
import pandas as pd
import os, sys
import matplotlib.pyplot as plt

# Command Line
cwd_0 = os.getcwd()
print('Initial Directory: ', cwd_0)

# Settings
plt.style.use('fivethirtyeight')

Initial Directory:  /Users/nickbenelli/Workspace/GitHub/Time-Series-Methods


In [2]:
from plot import colors

In [3]:
# Global Variables
plot_colors = colors.colors

param_name = 'Ozone'

init_color = "#008fd5"

from pylab import rcParams
int_figsize = rcParams['figure.figsize']


season = 52
test_year_start = 2022
n_splits = 3
use_normalized = False

## Explore Data
### Load Data

In [4]:
# Load Data
df_import = pd.read_csv("./daily_pollutants_by_county.csv")
df_poll0 = df_import.copy()
df_poll0['datetime'] = pd.to_datetime(df_poll0['datetime'])

df = df_poll0.loc[ df_poll0['county_code'] == 37 ]
df.index = df['datetime']
df = df.drop(columns= ['datetime'])
df = df.resample('W').mean()

time_series = df['aqi_O3']

time_series


/var/folders/tj/_56ljrb50_9gl48mcjz2fh4c0000gn/T/ipykernel_93268/2338816510.py:2: DtypeWarning: Columns (4,6,8,10,12,14,16,18,20,22) have mixed types. Specify dtype option on import or set low_memory=False.
  df_import = pd.read_csv("./daily_pollutants_by_county.csv")
/var/folders/tj/_56ljrb50_9gl48mcjz2fh4c0000gn/T/ipykernel_93268/2338816510.py:9: FutureWarning: The default value of numeric_only in DataFrameGroupBy.mean is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  df = df.resample('W').mean()


datetime
2012-01-01    27.928571
2012-01-08    21.683673
2012-01-15    19.061565
2012-01-22    25.614286
2012-01-29    24.104762
                ...    
2023-10-15    52.738095
2023-10-22    64.507937
2023-10-29    43.780423
2023-11-05    58.965608
2023-11-12    44.555556
Freq: W-SUN, Name: aqi_O3, Length: 620, dtype: float64

In [5]:
exog_feature_names = [
    'daily_meas_CO', 'daily_meas_Pb',
    'daily_meas_SO2', 'daily_meas_PM10', 'daily_meas_PM2.5', 
    'daily_meas_O3', 'daily_meas_NO2'
]

df_exog = df[exog_feature_names]
df_exog = df_exog.interpolate().fillna(df_exog.mean())


### Test Train Split


In [6]:
train_test_filter = time_series.index.year < test_year_start

train_time_series = time_series.loc[ train_test_filter ]
test_time_series = time_series.loc[ ~(train_test_filter) ]


print("X Shape: ", train_time_series.shape)
print("y Shape: ", test_time_series.shape)

print("test percent: ", round(len(test_time_series) / (len(test_time_series)+ len(train_time_series)), 2))


df_exog_train = df_exog.loc[ train_test_filter ]
df_exog_test = df_exog.loc[ ~(train_test_filter) ]

X Shape:  (522,)
y Shape:  (98,)
test percent:  0.16


## Time Series forecasting 
[Statsmodel example notebooks](https://github.com/statsmodels/statsmodels/tree/master/examples/notebooks)

__K Folds Cross Validation__

In [7]:
from sklearn.model_selection import TimeSeriesSplit

tss = TimeSeriesSplit(n_splits = n_splits)
print(tss)

tr_idx_list = []
val_idx_list = []
for i, (train_index, test_index) in enumerate(tss.split(train_time_series)):
    print(f"Fold {i}:")
    print(f"  Train: index={train_index.shape}")
    print(f"  Test:  index={test_index.shape}")
    tr_idx_list.append(train_index)
    val_idx_list.append(test_index)

print("Trian Test Idx Lengths:", len(tr_idx_list), len(val_idx_list))

TimeSeriesSplit(gap=0, max_train_size=None, n_splits=3, test_size=None)
Fold 0:
  Train: index=(132,)
  Test:  index=(130,)
Fold 1:
  Train: index=(262,)
  Test:  index=(130,)
Fold 2:
  Train: index=(392,)
  Test:  index=(130,)
Trian Test Idx Lengths: 3 3


------------------------
## Model Options
After doing some intial testing with the pmdarima package we are looking into the following models
Models to Try:

    SARIMA(2,0,2)x(3,1,0)[52]
    
    SARIMA(1,0,2)x(3,1,0)[52]


In [8]:
model_outcome_list = []
accuracy_list = []

#### Model #1: 

In [10]:
from statsmodels.tsa.statespace.sarimax import SARIMAX

In [106]:
my_order = (1, 0, 1)
my_seasonal_order = (3, 0, 0, season)


In [113]:
import flatdict

In [118]:
import evaluator
import importlib
importlib.reload(evaluator)
import evaluator
import accuracy
importlib.reload(accuracy)


<module 'accuracy' from '/Users/nickbenelli/Workspace/GitHub/Time-Series-Methods/accuracy.py'>

In [119]:
def forecast_model(model, steps =None,df_exog_test=None):
    """
    Forecast using a fitted SARIMAX model.

    Parameters:
    - model: Fitted SARIMAX model.
    - steps: Number of steps to forecast.

    Returns:
    - fcast = model.get_forecast()
    """
    # Forecast
    fcast = model.get_forecast(steps=steps, exog=df_exog_test,  print_result= False)
    data_pred = fcast.predicted_mean
    ci = fcast.conf_int()
    ci.index = fcast.predicted_mean.index
    fcast.ci = ci
    fcast.pred_data = data_pred
    fcast.pred_vals = data_pred.values
    return fcast

In [120]:
accuracy_list = []
forecast_results = []
models = []
use_exogen = True


In [121]:



for k in range(n_splits):
    print('----------------------------------------------')
    print(f"k={k}")

    # tr, val data
    tr_iloc_idx = tr_idx_list[k]
    val_iloc_idx = val_idx_list[k]

    # Subset Data
    tr_data = train_time_series.iloc[tr_iloc_idx]
    val_data = train_time_series.iloc[val_iloc_idx]

    df_exog_tr = df_exog_train.iloc[tr_iloc_idx]
    df_exog_val = df_exog_train.iloc[val_iloc_idx]

    # Model Fit
    model = SARIMAX(tr_data, exog=df_exog_tr, order=my_order, seasonal_order=my_seasonal_order)
    model_result = model.fit()



    forecast_step = len(val_data)
    fcast = forecast_model(model_result, steps= forecast_step, df_exog_test= df_exog_val)
    forecast_results.append(fcast)


    # Evaluate Data
    ModelEvaluator = evaluator.SARIMAXEvaluator(model_result)
    model_acc_map =ModelEvaluator.train_and_test_accurcy_metrics(val_data, test_exog_data= df_exog_val)
    model_acc_map['k'] = k
    k_model_name = f"{ModelEvaluator.model_name}-k:{k}"
    model_acc_map['model_name'] = k_model_name
    model_acc_map['model'] = model
    model_acc_map['model_result'] = model_result
    acc_map_flat = evaluator.flatten_acc_maps(model_acc_map, delimiter='.'))

    # Append to lists
    models.append(model_result)
    accuracy_list.append(acc_map_flat)





----------------------------------------------
k=0


/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/statsmodels/tsa/statespace/sarimax.py:866: UserWarning: Too few observations to estimate starting parameters for seasonal ARMA. All parameters except for variances will be set to zeros.
  warn('Too few observations to estimate starting parameters%s.'
 This problem is unconstrained.


RUNNING THE L-BFGS-B CODE

           * * *

Machine precision = 2.220D-16
 N =           13     M =           10

At X0         0 variables are exactly at the bounds

At iterate    0    f=  2.77215D+00    |proj g|=  8.65822D-02

At iterate    5    f=  2.77059D+00    |proj g|=  2.24801D-01

At iterate   10    f=  2.76728D+00    |proj g|=  1.78823D-02

At iterate   15    f=  2.76653D+00    |proj g|=  1.07138D-01

At iterate   20    f=  2.76218D+00    |proj g|=  1.80466D-01

At iterate   25    f=  2.75563D+00    |proj g|=  3.48925D-02

At iterate   30    f=  2.74379D+00    |proj g|=  1.78013D-02

At iterate   35    f=  2.70341D+00    |proj g|=  2.18543D-01

At iterate   40    f=  2.67635D+00    |proj g|=  3.78515D-02

At iterate   45    f=  2.66526D+00    |proj g|=  4.38991D-02


/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/statsmodels/base/model.py:607: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  warnings.warn("Maximum Likelihood optimization failed to "



At iterate   50    f=  2.66054D+00    |proj g|=  3.89567D-03

           * * *

Tit   = total number of iterations
Tnf   = total number of function evaluations
Tnint = total number of segments explored during Cauchy searches
Skip  = number of BFGS updates skipped
Nact  = number of active bounds at final generalized Cauchy point
Projg = norm of the final projected gradient
F     = final function value

           * * *

   N    Tit     Tnf  Tnint  Skip  Nact     Projg        F
   13     50     58      1     0     0   3.896D-03   2.661D+00
  F =   2.6605379994121847     

STOP: TOTAL NO. of ITERATIONS REACHED LIMIT                 


...

/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/statsmodels/tsa/statespace/representation.py:374: FutureWarning: Unknown keyword arguments: dict_keys(['print_result']).Passing unknown keyword arguments will raise a TypeError beginning in version 0.15.
  warnings.warn(msg, FutureWarning)
/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/statsmodels/tsa/statespace/representation.py:374: FutureWarning: Unknown keyword arguments: dict_keys(['print_result']).Passing unknown keyword arguments will raise a TypeError beginning in version 0.15.
  warnings.warn(msg, FutureWarning)


----------------------------------------------
k=2
RUNNING THE L-BFGS-B CODE

           * * *

Machine precision = 2.220D-16
 N =           13     M =           10

At X0         0 variables are exactly at the bounds

At iterate    0    f=  2.85653D+00    |proj g|=  4.74211D-01


 This problem is unconstrained.



At iterate    5    f=  2.83231D+00    |proj g|=  2.95867D-02

At iterate   10    f=  2.81127D+00    |proj g|=  6.93711D-02

At iterate   15    f=  2.79194D+00    |proj g|=  4.54725D-02

At iterate   20    f=  2.76916D+00    |proj g|=  1.31722D-02

At iterate   25    f=  2.76558D+00    |proj g|=  7.07738D-03

At iterate   30    f=  2.76178D+00    |proj g|=  2.02739D-03


KeyboardInterrupt: 

In [ ]:
pd.DataFrame(accuracy_list)

,train.mape,train.mae,train.mpe,train.rmse,train.corr,train.minmax,train.R_score,train.R2_corr,train.aic,train.bic,...,test.minmax,test.R_score,test.R2_corr,test.aic,test.bic,test.llf,test.model,test.type,k,model_name
0,0.052,2.413,0.007,3.595,0.972,0.049,0.943,0.943,728.38,765.86,...,0.083,0.876,0.876,728.38,765.86,-351.19,"SARIMA(1,0,1)x(3,0,0)[52]",predict,0,"SARIMA(1,0,1)x(3,0,0)[52]-k:0"
1,0.054,2.679,0.013,3.836,0.969,0.051,0.936,0.936,1461.82,1508.21,...,0.094,0.837,0.837,1461.82,1508.21,-717.91,"SARIMA(1,0,1)x(3,0,0)[52]",predict,1,"SARIMA(1,0,1)x(3,0,0)[52]-k:1"


In [ ]:
# Plot
line_pred_style = '--'

plt.figure(figsize=(16, 16))

for k in range(n_splits):



for k in range(n_splits):
  plt.subplot(n_splits, 1, (k+1))

  # tr, val data
  tr_iloc_idx = tr_idx_list[k]
  val_iloc_idx = val_idx_list[k]

  # Subset Data
  tr_data = train_time_series.iloc[tr_iloc_idx]
  val_data = train_time_series.iloc[val_iloc_idx]

  df_exog_tr = df_exog_train.iloc[tr_iloc_idx]
  df_exog_val = df_exog_train.iloc[val_iloc_idx]

  
  plt.plot(pd.concat([tr_data, val_data], axis=0), marker='o', markersize=3, linewidth=2, label= 'true')

  # Model 
  model = models[k]
  model_name = model_name

  plt.plot(model.fittedvalues, marker='o', markersize=2, linewidth=3,  label= f'{model_name} fitted')
  plt.plot(k_fold_map['pred_data'][k], line_pred_style, markersize=3, linewidth=2, label= f'{model_name} pred')

  ci = k_fold_map['ci'][k]

  plt.fill_between(ci.index, ci.iloc[:, 0], ci.iloc[:, 1], alpha=0.1, label = f'{model_name} pred ci')
  plt.title(f'{model_name} Val k={k}')
  if (k+1) == n_splits:
    plt.xticks(rotation=45, ha='right')
    plt.xlabel('Time (Weeks)')
  plt.ylabel(metric_um)

    
plt.show()

In [ ]:
model_outcome_list.append(k_fold_map)
k_fold_map['accuracy'][-1]
pd.DataFrame([k_fold_map['accuracy'][-1]])

#### R Squared Values 

In [ ]:
# Correlated
#model.fittedvalues.name = 'Fitted'
df_true_v_fitted = pd.concat([data_tr_k, model.fittedvalues], axis=1).iloc[season:]
display(df_true_v_fitted.head(5))
display(df_true_v_fitted.corr() ** 2)
fitted_r2 = (df_true_v_fitted.corr() ** 2).iloc[1,0]
print(f"Model ({model_name}) Fitted R^2: {fitted_r2}")




df_true_v_forecast = pd.concat([data_val_k, data_pred_k], axis=1)
display(df_true_v_forecast.head(5))
display(df_true_v_forecast.corr() ** 2)
forecast_r2 = (df_true_v_forecast.corr() ** 2).iloc[1,0]
print(f"Model ({model_name}) Forecast R^2: {forecast_r2}")


#### PMDARIMA Model Param Check

In [ ]:
import pmdarima as pm 

pm_model = pm.arima.ARIMA(order=my_order, seasonal_order=my_seasonal_order).fit(data_tr_k)

pm_model.summary()

___Residual Analysis Model 1___

In [ ]:
model.plot_diagnostics(figsize=(14, 10))
plt.show()

______
### Compare Accuracies


In [ ]:
pd.DataFrame(accuracy_list)

---------------------------
## Best Model
Pick the best model
- Use all training data and compare it to test data

In [ ]:
# Model Parameters
my_order = (2, 0, 2)
my_seasonal_order  = (3, 1, 0, season)

my_order_str = str(my_order).replace(' ', '')
my_seasonal_order_str = str(my_seasonal_order[0:3]).replace(' ', '')

model_name = f"SARIMA{my_order_str}x{my_seasonal_order_str}[{my_seasonal_order[3]}]"

print(model_name)

In [ ]:
k_fold_map = {
    'k' : [],
    'train_idx' : [],
    'val_idx' : [],
    'train_data' : [],
    'val_data' : [], 
    'ytrue' : [], 

    'model' : [], 
    'model_name' : [], 

    'pred_data' : [],
    'ypred' : [],
    'ci' : [], 
    'accuracy' : [], 
}

df_X = pd.DataFrame(X_data)
df_y = pd.DataFrame(y_data)
#k = 0




# train, test data
#train_idx = train_idx_list[k]
#test_idx = test_idx_list[k]
#data_tr_k, data_val_k = df_X.iloc[train_idx, :], df_X.iloc[test_idx, :]
y_true = df_y.values

# Model Fit
model = SARIMAX(df_X, order=my_order, seasonal_order=my_seasonal_order).fit()
#k_model_name = f"{model_name}"

data_pred = model.forecast(len(df_y))
fcast = model.get_forecast(len(df_y))
ci = fcast.conf_int()
y_pred = data_pred.values

acc_dict = epa.ts_methods.accuracy.find_prediction_acc(y_pred, y_true, print_result=False)
acc_dict['aic'] = round(model.aic, 2)
acc_dict['bic'] = round(model.bic, 2)
acc_dict['llf'] = round(model.llf, 2)
acc_dict['model'] = model_name
acc_dict['k'] = 'train'

k_fold_map['k'].append('train')
k_fold_map['train_idx'].append(None)
k_fold_map['val_idx'].append(None)
k_fold_map['train_data'].append(X_data)
k_fold_map['val_data'].append(y_data)
k_fold_map['ytrue'].append(y_true)
# Model
k_fold_map['model'].append(model)
k_fold_map['model_name'].append(k_model_name)
# Predction
k_fold_map['pred_data'].append(data_pred)
k_fold_map['ypred'].append(y_pred)
k_fold_map['ci'].append(ci)
k_fold_map['accuracy'].append(acc_dict)



In [ ]:
accuracy_list.append(acc_dict)
pd.DataFrame([acc_dict]).T

In [ ]:
model_outcome_list.append(k_fold_map)
k_fold_map['accuracy'][-1]
pd.DataFrame([k_fold_map['accuracy'][-1]]).T

In [ ]:
# Plot
line_pred_style = '--'

plt.figure(figsize=(18, 8))


plt.plot(pd.concat([X_data, y_data], axis=0), marker='o', markersize=3, linewidth=2, label= 'true')

# Model 
#model = k_fold_map['model'][k]
#model_name = k_fold_map['model_name'][k]

plt.plot(model.fittedvalues, marker='o', markersize=2, linewidth=3,  label= f'{model_name} fitted')
plt.plot(data_pred, line_pred_style, markersize=3, linewidth=2, label= f'{model_name} pred')


plt.fill_between(ci.index, ci.iloc[:, 0], ci.iloc[:, 1], alpha=0.1, label = f'{model_name} pred ci')
#plt.title(f'Model Comparison Cross Val k={k}')
if use_normalized:
    plt.title(f'{param_name} Model: {model_name} Forecast (Normalized)')
else:
    plt.title(f'{param_name} Model: {model_name} Forecast')
plt.xticks(rotation=45, ha='right')
plt.xlabel('Time (Weeks)')
plt.legend()
plt.ylabel(metric_um)

    
plt.show()

In [ ]:
print("Accuracy:")
pd.DataFrame(accuracy_list)

__Inverse Normalized Points__
    
If the data is normalized inverse the normalization to true points

In [ ]:
if use_normalized:
    print("Inversing Normalization")
    # Fitted Values
    model_fit_vals_array = scaler.inverse_transform(pd.DataFrame(model.fittedvalues.copy()))
    model_fitted_vals_inv_series = pd.Series(model_fit_vals_array.flatten(), index= model.fittedvalues.index, name=model.fittedvalues.name)

    # Predicted Values
    data_pred_array = scaler.inverse_transform(pd.DataFrame(data_pred.copy()))
    data_pred_inv = pd.Series(data_pred_array.flatten(), index= data_pred.index, name=data_pred.name)

    # CI
    ci_inv = pd.DataFrame(scaler.inverse_transform(ci.copy()), index=ci.index, columns=ci.columns)

    # Assigned Names
    fitted_vals = model_fitted_vals_inv_series.copy()
    pred_vals = data_pred_inv.copy()
    ci_vals = ci_inv.copy()
else:
    print("No Inverse Needed")
    fitted_vals = model.fittedvalues.copy()
    pred_vals = data_pred.copy()
    ci_vals = ci.copy()
    #model_fitted_vals_inv_series = model.fittedvalues
    
    

In [ ]:
# Plot
line_pred_style = '--'

plt.figure(figsize=(18, 8))


plt.plot(pd.concat([X, y], axis=0), marker='o', markersize=3, linewidth=2, label= 'true')

# Model 
#model = k_fold_map['model'][k]
#model_name = k_fold_map['model_name'][k]

plt.plot(fitted_vals, marker='o', markersize=2, linewidth=3,  label= f'{model_name} fitted')
plt.plot(pred_vals, line_pred_style, markersize=3, linewidth=2, label= f'{model_name} pred')


plt.fill_between(ci_vals.index, ci_vals.iloc[:, 0], ci_vals.iloc[:, 1], alpha=0.1, label = f'{model_name} pred ci')
#plt.title(f'Model Comparison Cross Val k={k}')
plt.title(f'{param_name} Model: {model_name} Forecast')
plt.xticks(rotation=45, ha='right')
plt.xlabel('Time (Weeks)')
plt.legend()
plt.ylabel(metric_um)

    
plt.show()

In [ ]:
# Forecast vs y true

plt.figure(figsize=(14, 6))
plt.plot(pd.concat([y], axis=0), marker='o', label= 'true')
#plt.plot(model.fittedvalues, marker='o', label= 'fitted')
plt.plot(data_pred, line_pred_style, marker='o', label= 'pred')
plt.fill_between(ci.index, ci.iloc[:, 0], ci.iloc[:, 1], alpha=0.1, label = f'{model_name} pred ci')
#plt.fill_between(ci.index, ci['lower cases'], ci['upper cases'], color='blue', alpha=0.1, label = f'pred ci')
plt.title(f'{param_name}: {model_name} Prediction')

plt.xticks(rotation=45, ha='right')
plt.xlabel('Weeks')
plt.ylabel(metric_um)
plt.legend()
plt.tight_layout()
plt.show()

In [ ]:
pd.DataFrame([acc_dict]).T

__R Squared Values__

In [ ]:
# Correlated
#model.fittedvalues.name = 'Fitted'
df_true_v_fitted = pd.concat([X_data, model.fittedvalues], axis=1).iloc[season:]
display(df_true_v_fitted.head(3))

display(df_true_v_fitted.corr() ** 2)
fitted_r2 = (df_true_v_fitted.corr() ** 2).iloc[1,0]
print(f"Model ({model_name}) Fitted R^2: {fitted_r2}")



df_true_v_forecast = pd.concat([y_data, pred_vals], axis=1)
display(df_true_v_forecast.head(3))
display(df_true_v_forecast.corr() ** 2)
forecast_r2 = (df_true_v_forecast.corr() ** 2).iloc[1,0]
print(f"Model ({model_name}) Forecast R^2: {forecast_r2}")


print('\n\n')
print(f"Model ({model_name}) Fitted R^2:\t {fitted_r2}")
print(f"Model ({model_name}) Forecast R^2:\t {forecast_r2}")


__PMDARIMA Model Param Check__

In [ ]:
import pmdarima as pm 

pm_model = pm.arima.ARIMA(order=my_order, seasonal_order=my_seasonal_order).fit(data_tr_k)

pm_model.summary()

### Training Data Resiudal Analysis

In [ ]:
print(model_name)
display(model.summary())

In [ ]:
print(f'\n{model_name}')
model.plot_diagnostics(figsize=(14, 10))
plt.show()

In [ ]:
import statsmodels.api as sm

if my_seasonal_order[1] > 0:
    model_residuals = model.resid[(season * my_seasonal_order[1]):]
else:
    model_residuals = model.resid


df_model_residuals  = pd.DataFrame(model_residuals)
display(df_model_residuals.describe())

#df_model_residuals.plot()
#df_model_residuals.plot(kind='kde')
#plt.title(f'ARIMA{pdqParmas_list[inc]} Residuals')
#plt.show()

# Plot Resiudals
ts_fig_size = (10, 4)
acf_pacf_fig_size = (12,4)

#Plot Time Series
plt.figure(figsize= ts_fig_size, dpi=80)
plt.plot( df_model_residuals, marker='o')
plt.title(f'{model_name} Residual Time Series')
plt.xticks(rotation=45, ha='right')
plt.xlabel('Time (Weeks)')
plt.ylabel('Residual')
plt.show()


# acf pacf plts
fig, ax = plt.subplots(1,2,figsize= acf_pacf_fig_size)
# acf
sm.graphics.tsa.plot_acf(df_model_residuals.values.squeeze(), lags=35, ax=ax[0], title=f'{model_name} Resid ACF')
# pacf
sm.graphics.tsa.plot_pacf(df_model_residuals, lags=35, ax=ax[1], title=f'{model_name} Resid pACF') 
plt.tight_layout()
plt.show()

In [ ]:
import scipy.stats as stats
fig = plt.figure(figsize = (8, 8))

ax = fig.add_subplot(1,1, 1)
res = stats.probplot(model_residuals, plot=plt)


#ax.get_lines()[0].set_marker('o')
#ax.get_lines()[0].set_markerfacecolor(line_color_list[inc])
#ax.get_lines()[0].set_markersize(4)

ax.get_lines()[1].set_linewidth(5)
plt.title(f'{model_name} QQ-Plot', fontsize=16)
plt.tight_layout()
plt.show()


print(f'\n\n{model_name}')
epa.ts_methods.shapiro_wilk_test(model.resid)


In [ ]:

#Ljung Box Test

fig = plt.figure(figsize = (12, 4))
print('\n\n')
print(f'{model_name}')
lbt_array = sm.stats.acorr_ljungbox(pd.DataFrame(model_residuals), lags=30)
display(lbt_array.tail(1))

#plt.plot(lbt_array.lb_pvalue, marker='o', markersize=5, linewidth=0)
plt.plot(lbt_array.lb_pvalue, marker='o', markersize=5, linewidth=0, label= f'{model_name}')
plt.xlabel('Lag')
plt.ylabel('p-value')
plt.title(f'Ljung-Box Test')
plt.legend()
plt.show()
     

In [ ]:
import seaborn as sns

# Standardize Residuals
#fig = plt.figure(figsize = (12, 4))
sqr_dev_mean = ((model_residuals - model_residuals.mean())**2).sum()
ar_resid_leverage = 1/model_residuals.count()+(model_residuals- model_residuals.mean())**2 / sqr_dev_mean

ar_std_residuals = model_residuals/ (model_residuals.std() * (1-ar_resid_leverage)**(1/2))

#ax = fig.add_subplot(1,2, inc+1)
sns.distplot(ar_std_residuals) # you may select the no. of bins
#sns.histplot(ar_std_residuals) # you may select the no. of bins
plt.title(f'Standardize Error Terms {model_name}', fontsize=16)           
plt.xlabel('Residuals', fontsize = 15) 
#plt.tight_layout()    
plt.show()